In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
# import tensorflow as tf

# def get_session():
#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     return tf.Session(config=config)

# use this environment flag to change which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# set the modified tf session as backend in keras
# keras.backend.tensorflow_backend.set_session(get_session())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import csv

In [ ]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = './pred_overfit.h5'

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.load_model(model_path, backbone_name='resnet50', convert_model=True)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'lice'}

In [ ]:
# load image
img_path = '/root/data/lice_detection/1_fish_classification/lice_fish/fish_226.jpg'
image = read_image_bgr(img_path)

# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)

# process image
start = time.time()
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
print("processing time: ", time.time() - start)

In [ ]:
boxes[0][0] / 0.4

In [ ]:
# find ground truth
dataset = []
with open('/root/data/lice_detection/lice_dataset_fish_only_train.csv', 'r') as csvfile:
    reader = csv.reader(csvfile) #, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for row in reader:
        dataset.append(row)

ground_truth = []
for d in dataset:
    if d[0] == img_path:
        ground_truth.append(d)

In [ ]:
# correct for image scale
boxes /= scale

In [ ]:
# visualize detections
fig,ax = plt.subplots(1, figsize=(15, 15))
im = np.array(Image.open(img_path), dtype=np.uint8)
ax.imshow(im)
for (i, (box, score, label)) in enumerate(zip(boxes[0], scores[0], labels[0])):
    # scores are sorted so we can break
    if score < 0.1:
        break
    color = label_color(label)
    b = box.astype(int)
    # draw_box(draw, b, color=color)
    if i == 0:
        rect = patches.Rectangle((b[0],b[1]),b[2]-b[0],b[3]-b[1],
                                 linewidth=1,edgecolor='b',facecolor='none', label='prediction')
    else:
        rect = patches.Rectangle((b[0],b[1]),b[2]-b[0],b[3]-b[1],
                         linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)
    # caption = "{} {:.3f}".format(labels_to_names[label], score)
    # draw_caption(draw, b, caption)


# Create a Rectangle patch
for (i, gt) in enumerate(ground_truth):
    rec = [int(d) for d in gt[1:5]]
    # print(rec)
    if i == 0:
        rect = patches.Rectangle((rec[0],rec[1]),rec[2]-rec[0],rec[3]-rec[1],
                                 linewidth=1,edgecolor='r',facecolor='none', label='ground truth')
    else:
        rect = patches.Rectangle((rec[0],rec[1]),rec[2]-rec[0],rec[3]-rec[1],
                                 linewidth=1,edgecolor='r',facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)

plt.legend()
plt.show()
    
    
# plt.figure(figsize=(15, 15))
# plt.axis('off')
# plt.imshow(draw)
# plt.show()

# display ground truth

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import csv

In [ ]:
img_path = '/root/data/lice_detection/1_fish_classification/lice_fish/fish_226.jpg'

In [ ]:
dataset = []
with open('/root/data/lice_detection/lice_dataset_fish_only_train.csv', 'r') as csvfile:
    reader = csv.reader(csvfile) #, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for row in reader:
        dataset.append(row)

In [ ]:
print(len(dataset))

In [ ]:
ground_truth = []
for d in dataset:
    if d[0] == img_path:
        ground_truth.append(d)

In [ ]:
im = np.array(Image.open(img_path), dtype=np.uint8)

# Create figure and axes
fig,ax = plt.subplots(1)

# Display the image
ax.imshow(im)

# Create a Rectangle patch
for gt in ground_truth:
    rec = [int(d) for d in gt[1:5]]
    # print(rec)
    rect = patches.Rectangle((rec[0],rec[1]),rec[2]-rec[0],rec[3]-rec[1],linewidth=1,edgecolor='r',facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

plt.show()